In [1]:
from duckduckgo_search import DDGS

In [ ]:
# import os
# from typing import Any, Text, Dict, List
# from duckduckgo_search import DDGS  # pip install duckduckgo-search
# from openai import OpenAI  # pip install openai

# from rasa_sdk import Action, Tracker
# from rasa_sdk.executor import CollectingDispatcher
# from rasa_sdk.events import SlotSet


class ActionGiveCareerAdvice(Action):
    def name(self) -> Text:
        return "action_give_career_advice"

    def run(
        self,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: Dict[Text, Any],
    ) -> List[Dict[Text, Any]]:

        # Get slots
        major = tracker.get_slot("current_major") or "your major"
        interest = tracker.get_slot("career_interest") or "careers"
        year = tracker.get_slot("year_of_study") or "your year"
        gpa = tracker.get_slot("gpa") or "strong performance"  # Optional
        has_internship = tracker.get_slot("has_internship") or False

        # Build search query (contextual + timely)
        query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"
        
        try:
            # Search DuckDuckGo (top 3 results)
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=3))
            
            # Extract snippets & sources
            snippets = [r["body"] for r in results]
            sources = [r["href"] for r in results]
            sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])
            
            # LLM prompt for friendly summary
            client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
            prompt = f"""
            You are a warm, encouraging university career counselor. Summarize these search results into 4-5 bullet points of personalized advice for a {year} {major} student interested in {interest}.
            Keep it upbeat, use simple language, add 1-2 emojis. End with a nudge to build skills or network.
            Focus on jobs, skills, internships, resume tips. Base ONLY on these snippets – no external knowledge.
            
            Snippets:
            {chr(10).join(snippets)}
            """
            
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # Cheap & fast
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=300
            )
            advice_text = response.choices[0].message.content.strip()
            
        except Exception as e:
            # Fallback if search/LLM fails
            advice_text = f"""
            Hey, pulling live info can be tricky sometimes! Here's timeless advice for {interest} in {major}:
            - **Explore roles** like entry-level analyst or developer – demand is booming! 🚀
            - **Build skills** with free Coursera courses in Python or data viz.
            - **Internships**: Check Handshake for summer spots; start applying now.
            - **Resume tip**: Quantify wins, e.g., "Led team project saving 20% time."
            Remember, every expert started as a student like you! 💪
            """
            sources_list = "Fallback: General career resources (e.g., your school's site)."
        
        # Utter advice
        dispatcher.utter_message(
            response="utter_advice_core",
            advice_text=advice_text,
            current_major=major,
            career_interest=interest,
            year_of_study=year,
        )
        dispatcher.utter_message(response="utter_encourage_human")
        dispatcher.utter_message(
            response="utter_cite_sources",
            sources_list=sources_list
        )
        
        return []

In [2]:
# Get slots
major = "your major"
interest = "careers"
year = "your year"
gpa = "strong performance"  # Optional
has_internship = False

# Build search query (contextual + timely)
query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"

try:
    # Search DuckDuckGo (top 3 results)
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
    
    # Extract snippets & sources
    snippets = [r["body"] for r in results]
    sources = [r["href"] for r in results]
    sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])
    
    # # LLM prompt for friendly summary
    # client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    # prompt = f"""
    # You are a warm, encouraging university career counselor. Summarize these search results into 4-5 bullet points of personalized advice for a {year} {major} student interested in {interest}.
    # Keep it upbeat, use simple language, add 1-2 emojis. End with a nudge to build skills or network.
    # Focus on jobs, skills, internships, resume tips. Base ONLY on these snippets – no external knowledge.
    
    # Snippets:
    # {chr(10).join(snippets)}
    # """
    
    # response = client.chat.completions.create(
    #     model="gpt-4o-mini",  # Cheap & fast
    #     messages=[{"role": "user", "content": prompt}],
    #     temperature=0.3,
    #     max_tokens=300
    # )
    # advice_text = response.choices[0].message.content.strip()
    
except Exception as e:
    # Fallback if search/LLM fails
    advice_text = f"""
    Hey, pulling live info can be tricky sometimes! Here's timeless advice for {interest} in {major}:
    - **Explore roles** like entry-level analyst or developer – demand is booming! 🚀
    - **Build skills** with free Coursera courses in Python or data viz.
    - **Internships**: Check Handshake for summer spots; start applying now.
    - **Resume tip**: Quantify wins, e.g., "Led team project saving 20% time."
    Remember, every expert started as a student like you! 💪
    """
    sources_list = "Fallback: General career resources (e.g., your school's site)."

# # Utter advice
# dispatcher.utter_message(
#     response="utter_advice_core",
#     advice_text=advice_text,
#     current_major=major,
#     career_interest=interest,
#     year_of_study=year,
# )
# dispatcher.utter_message(response="utter_encourage_human")
# dispatcher.utter_message(
#     response="utter_cite_sources",
#     sources_list=sources_list
# )

/tmp/ipykernel_21474/1277786559.py:13: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [5]:
from ddgs import DDGS

# Perform a text search
results = DDGS().text("python programming", max_results=10)


snippets = [r["body"] for r in results]
sources = [r["href"] for r in results]
sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])

print(snippets)
print(sources_list)

['Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Beginning with Python 3.5, capabilities and keywords for typing were added to language, allowing optional static typing. Currently only versions in the 3.x series are supported. Python has gained widespread use in the machine learning community. It is widely taught as an introductory programming language. Since 2003, Python has consistently ranked in the top ten of the most popular programming languages in the TIOBE Programming 

In [6]:
from ddgs import DDGS

# Example variables
interest = "data science"
major = "computer science"
year = "junior"
gpa = "3.8"
has_internship = True

# Build query
query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"

# Perform search
results = DDGS().text(query, max_results=10)
for result in results:
    print(result["body"], result["href"])   

Train For A New Career In IT With A Data Science Traineeship And A Guaranteed Placement. Start Your New Career Today With A Guaranteed Job After Training Or Your Course Fees Back. https://duckduckgo.com/y.js?ad_domain=itcareerswitch.co.uk&ad_provider=bingv7aa&ad_type=txad&click_metadata=UZcNSwZ4ZrYXkhyeh4Rur-oBXOgH0p5tQNn7b1dSpOvkFOsWuRtmwjQl152Lykak9V36RQTG_sYWTFZv3gBbjlKwUQrCHTRn1lZP_lS_GDQEyUUbln4sFX-hjQOqmCGFrXHXLhFnR9qE5FcYSnNEeg.9x4qmsYv6Mq1XNntacbWMA&rut=dceeb07233da1ed2d95ed79f7b2957de579941c4f424589fa6e13ff384d7b776&u3=https://www.bing.com/aclick?ld=e8MwuUYDzl9PotoF1qnly1-DVUCUxs9NcB9xxf6-0FQaCqZkI6uUpTcIz5u0vTDDvaN3Y-sEoJK00Sqk_JJgRLAw7A6PP2nMEs_Q1LvGURBjxkySaDtnYjbpiFVzXjynCYNJ7nitiTJ_p-QOkick_YzTiuhGLLNy_mDeVEjwQkoNEkRCRf8WvMBNChdVbDA40Xa-EuX5nnrpVcjXgB_cKqFhqIEEk&u=aHR0cHMlM2ElMmYlMmZpdGNhcmVlcnN3aXRjaC5jby51ayUyZmxhbmRpbmctYiUyZmRhdGEtc2NpZW5jZSUyZiUzZm1zY2xraWQlM2Q4ZTNiNjA4ODA0NDYxOTI4MWRjNDA5ODFlMWVlZjE5ZCUyNnV0bV9zb3VyY2UlM2RiaW5nJTI2dXRtX21lZGl1bSUzZGNwYyUyNnV0bV9jYW1

In [2]:
# from duckduckgo_ai_chat import DuckDuckGoAIChat
import duckduckgo_ai_chat.app as DAI

DAI.checkquery

# chat = DuckDuckGoAIChat()
# response = chat.ask("What are the latest trends in AI?")
# print(response)

<function duckduckgo_ai_chat.app.checkquery(args)>

In [4]:
import duckduckgo_ai_chat.app as app_module

# List all attributes in the package
print(dir(app_module))

# Additionally, you can check the module's help
help(app_module)


['DEBUG', 'Fore', 'MODELS', 'Queue', 'Style', 'Thread', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'accept_terms_of_service', 'checkquery', 'choose_model', 'datetime', 'fetch_response', 'fetch_vqd', 'get_download_folder', 'init', 'json', 'mainrun', 'os', 'print_banner', 'process_stream', 'random', 'requests', 'reset_stdin', 'save_session', 'sys', 'time']
Help on module duckduckgo_ai_chat.app in duckduckgo_ai_chat:

NAME
    duckduckgo_ai_chat.app

DESCRIPTION
    # /// script
    # dependencies = [
    #   "requests",
    #   "colorama",
    # ]
    # ///

FUNCTIONS
    accept_terms_of_service()

    checkquery(args)

    choose_model()

    fetch_response(chat_url, vqd, vqd_hash_1, model, messages)

    fetch_vqd()

    get_download_folder()

    mainrun(args)
        Inspired by duckduckGO-chat-cli

    print_banner()
        Print a colorful welcome banner

    process_stream(response, output_queue)

    reset_stdin()

 

In [4]:
from transformers import pipeline
import torch

# Load a model for text generation
generator = pipeline('text-generation', model='gpt2')
results = generator("Hello, how are you today?", max_length=50)
print(results)


NameError: name 'torch' is not defined

In [ ]:
import os
from google import genai

# Initialize the client with the API key from the environment variable
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words."
)

print(response.text)

In [2]:
import os

GEM_API_KEY=os.getenv("GEMS")

In [4]:
import google.generativeai as genai

# Configure the API key
genai.configure(api_key=GEM_API_KEY)

# Choose a model (e.g., 'gemini-pro' for text, 'gemini-pro-vision' for images)
model = genai.GenerativeModel('gemini-2.5-flash')

# Generate content
response = model.generate_content("Explain the theory of relativity in simple terms.")

print(response.text)

The theory of relativity, developed by Albert Einstein, is actually two interconnected theories: **Special Relativity** (1905) and **General Relativity** (1915). It's all about how space and time are not absolute and fixed, but are instead *relative* to the observer's motion and gravity.

Let's break it down into simple terms:

---

### The Core Idea: Everything is Relative (Except One Thing)

Imagine you're on a train.
*   If you roll a ball down the aisle, someone on the train sees it moving at, say, 5 mph.
*   But someone standing still on the platform sees the ball moving much faster (the speed of the train + 5 mph).
*   Both are "right" from their own perspective. That's the basic idea of *relativity* – motion depends on your viewpoint.

Einstein took this classic idea and added a huge twist: **The speed of light in a vacuum is always the same for everyone, no matter how fast they are moving.** This single, incredibly powerful idea leads to all the strange consequences of relativi

In [7]:
from ddgs import DDGS
import google.generativeai as genai

# Example variables
interest = "data science"
major = "computer science"
year = "junior"
gpa = "3.8"
has_internship = True


query = (
    f"2025 {interest} career advice {major} {year} student {gpa} GPA "
    f"{'with internship experience' if has_internship else 'entry-level'} "
    "site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"
)

results = list(DDGS().text(query, max_results=10))
snippets, sources = [], []
for result in results[:5]:
    snippets.append(result.get("body", ""))
    sources.append(result.get("href", ""))
sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources) if src])

# Compose the prompt
prompt = (
    f"You are a warm, encouraging university career counselor. Summarize these search results into 4-5 upbeat bullet points of advice for a {year} {major} student (GPA {gpa}) interested in {interest} "
    f"({'with internship experience' if has_internship else 'entry-level'}). "
    "Use simple language, add 1-2 emojis per point. Focus on 2025 jobs, skills, internships, and resume tips. "
    "Base ONLY on these snippets—no external knowledge. End with a networking nudge.\n\n"
    f"Snippets:\n{chr(10).join(snippets)}"
) # note: chr(10) <==> '\n'

GEM_API_KEY = os.getenv("GEMS")
genai.configure(api_key=GEM_API_KEY)

model = genai.GenerativeModel('gemini-2.5-flash')

response = model.generate_content(prompt)

advice_text = response.text

advice_text

"Hello there! It's wonderful to hear you're a junior in Computer Science with a fantastic GPA and already gaining internship experience – that's a brilliant start! Data science is an incredibly exciting field, and you're clearly on a great path. Based on the latest insights, here's some upbeat advice for you:\n\n*   **Strong Foundation for 2025 Jobs:** Your Computer Science degree is definitely a powerful asset for tech ambitions in 2025, providing a solid foundation for exploring career prospects and industry trends in data science. Keep building on that! 🚀💡\n*   **Explore Diverse Career Paths:** Data science professionals help companies improve and optimize business performance with their insights. Take some time to learn about the many different careers you can pursue in this field (there are at least 12!), which will help you make a strategic career decision. 📈🎯\n*   **Leverage Internships & Networking:** There are hundreds of Data Science Intern jobs available daily! Your existing

In [1]:
import os
import sys
from pathlib import Path
from typing import List, Tuple

# -------------------------------------------------
# 1️⃣  IMPORTS
# -------------------------------------------------
from ddgs import DDGS          # DuckDuckGo Search (renamed to "ddgs")
import google.generativeai as genai

# -------------------------------------------------
# 2️⃣  CONFIGURATION / USER INPUT
# -------------------------------------------------
interest = "cybersec"
major = "Info tech"
year = "senior"
gpa = "3.7"
has_internship = True

# -------------------------------------------------------------------------
# 3️⃣  BUILD A UK‑TARGETED SEARCH QUERY
# -------------------------------------------------------------------------
#  • We bias the engine toward the UK with region="uk-en"
#  • We also restrict the sites to UK domains so the results are truly local.
# -------------------------------------------------------------------------
query = (
    f"2025 {interest} career advice {major} {year} student {gpa} GPA "
    f"{'with internship experience' if has_internship else 'entry-level'} "
    # Site filters – feel free to add/remove
    "site:.ac.uk OR site:.gov.uk OR site:linkedin.com/company/uk- "
    "OR site:indeed.co.uk OR site:prospects.ac.uk OR site:targetjobs.co.uk"
)

# -------------------------------------------------------------------------
# 4️⃣  PERFORM THE SEARCH (UK‑biased)
# -------------------------------------------------------------------------
def fetch_snippets(
    query: str,
    max_results: int = 10,
    region: str = "uk-en",
) -> Tuple[List[str], List[str]]:
    """
    Returns:
        snippets – list of result bodies (plain text)
        sources  – list of result URLs (href)
    """
    try:
        # `region="uk-en"` tells DDGS to prioritize UK results.
        results = list(DDGS().text(query, max_results=max_results, region=region))
    except Exception as exc:
        print(f"[❗] DuckDuckGo search failed: {exc}", file=sys.stderr)
        return [], []

    snippets, sources = [], []
    for r in results[:5]:          # keep only the top‑5 most relevant
        snippets.append(r.get("body", ""))
        sources.append(r.get("href", ""))

    # Remove empty strings that may appear if a result has no body/href
    snippets = [s for s in snippets if s.strip()]
    sources  = [s for s in sources if s.strip()]

    return snippets, sources


snippets, sources = fetch_snippets(query)

# -------------------------------------------------------------------------
# 5️⃣  QUICK‑FAILSAFE: if we got no snippets, bail out with a friendly message.
# -------------------------------------------------------------------------
if not snippets:
    print("[⚠️] No search results were returned – try a broader query.")
    sys.exit(0)

# -------------------------------------------------------------------------
# 6️⃣  PREPARE A NICE SOURCE LIST (for the final answer)
# -------------------------------------------------------------------------
sources_list = "\n".join(
    f"[{i+1}] {src}" for i, src in enumerate(sources) if src
)

# -------------------------------------------------------------------------
# 7️⃣  BUILD THE GEMINI PROMPT (UK‑focused)
# -------------------------------------------------------------------------
prompt = (
    f"You are a warm, encouraging university career counsellor based in the United Kingdom. "
    f"Summarise the following search results into **4‑5 upbeat bullet points** of advice for a "
    f"{year} {major} student (GPA {gpa}) interested in {interest} "
    f"{'with internship experience' if has_internship else 'entry‑level'}.\n"
    f" • Use simple British English (no American slang).\n"
    f" • Add 1‑2 emojis per point.\n"
    f" • Focus on **2025 UK jobs, British employer graduate schemes, UK‑specific skills, internships, and CV/LinkedIn tips**.\n"
    f" • Base the answer **ONLY** on the snippets below – do NOT bring in outside knowledge.\n"
    f" • End with a short networking nudge that mentions UK‑centric platforms (e.g., LinkedIn, university alumni portals, or local meet‑ups).\n\n"
    f"Snippets:\n{chr(10).join(snippets)}"
)

# -------------------------------------------------------------------------
# 8️⃣  CONFIGURE GEMINI
# -------------------------------------------------------------------------
GEM_API_KEY = os.getenv("GEMS")
if not GEM_API_KEY:
    raise RuntimeError("❌ Environment variable 'GEMS' (Gemini API key) not set.")
genai.configure(api_key=GEM_API_KEY, transport='rest')  # ← transport='rest'

# Choose the model – `gemini-2.5-flash` is fast & cheap, perfect for short bullet‑list output.
model = genai.GenerativeModel('gemini-2.5-flash')

# -------------------------------------------------------------------------
# 9️⃣  CALL THE MODEL (with a small timeout & error handling)
# -------------------------------------------------------------------------
try:
    response = model.generate_content(
        prompt,
        request_options={"timeout": 30}  # ✅ Proper way to set timeout
    )
    advice_text = response.text.strip()
except Exception as exc:
    advice_text = f"Sorry, I couldn't generate the advice right now. ({exc})"

# -------------------------------------------------------------------------
# 10️⃣  COMBINE ADVICE + SOURCES (nice final formatting)
# -------------------------------------------------------------------------
final_output = f"{advice_text}\n\n**Sources**\n{sources_list}"
print(final_output)

/workspaces/ai-assistant/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello there! It's fantastic to hear you're a senior Info Tech student with a solid GPA and internship experience, especially with your sights set on the cyber security field. That's a brilliant area with so much growth! Here's some upbeat advice based on what we've seen:

*   **Harness your internship experience!** Your valuable IT work experience gained through an internship will truly set you apart from other candidates. It's a fantastic way to show British employers your passion for and understanding of the cyber security field, which they actively seek! ✨
*   **Bright 2025 UK Job Prospects!** You're entering a fast-growing field with a current skills shortage in the UK, meaning great opportunities for you! Leading British employers are aiming to recruit nearly 1,000 technology graduates in 2025, so keep an eye out for those openings. 📈
*   **Explore UK Graduate Schemes!** Many British employers offer excellent graduate schemes in cyber and information security. These programmes are